In [ ]:
import numpy as numpy
import pandas as pd

In [ ]:
movies=pd.read_csv('tmdb_5000_movies.csv')
movies.head()

In [ ]:
credits=pd.read_csv('tmdb_5000_credits.csv')
credits.head(5)

In [ ]:
merged_df=movies.merge(credits,on='title')
merged_df.shape


In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
merged_df.head()

## it is important for select tags


In [ ]:
merged_df.info()

# important tags
1. Genres
2. id
3. keywords
4. title
5. overviews
6. cast
7. crew
8. production_company

In [ ]:
movies=merged_df[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.head()

In [ ]:
movies.dropna(inplace=True)
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
'''[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "SciFictionence "}]'
= ['Action','Adventure','Fantasy','SciFictionence']'''

In [ ]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):  # obj is string so by using this code convert into list so we can change or perform  operation 
        L.append(i['name'])
    return L

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:

def convert3(obj):
    L=[]
    counter =0
    for i in ast.literal_eval(obj):  # obj is string so by using this code convert into list so we can change or perform  operation 
        if counter !=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies['cast']=movies['cast'].apply(convert3)


In [ ]:
movies.head()

In [ ]:
# movies['crew'][0]

In [ ]:

def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):  # obj is string so by using this code convert into list so we can change or perform  operation 
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
#   Sam Worthington = SamWorthington

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x]) # ScienceFiction (done)


In [ ]:
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x]) # ScienceFiction (done)


In [ ]:
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x]) # ScienceFiction (done)


In [ ]:
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x]) # ScienceFiction (done)

In [ ]:
movies.head()

In [ ]:
movies['tags']=movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew'] # merge row wise

In [ ]:
movies.head()

In [ ]:
print(movies.tags[0]) # merge row wise

In [ ]:
new_df=movies[['title','movie_id','tags',]]

In [ ]:
new_df.head()

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

## nltk library used
#### to ['loving','loved','love'] ==love in sb words ko ak he words ke equal assign krna 

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df['tags'][1]

# Conver text into vectors by vectorization
1. bag of words technique
2. tf idf tech
3. word to vec tech

In [ ]:
# tags 1 +tag 2 + tag3 .................5000 times
#          word 1+ word 2 +word 3 +   word 4................lakhs
# movie1 =  4        3       5           5  == 1 vector 
# movie2 =  1        1       4           0 === 1 vector
# movie3=   5         0      6           1 == 1 vector
# movie4=   5         0      6           1
# movie5=   5         0      6           1
#5000 times
#Extract 5000 words from lakhs words
#we are not include - are , is , am ,was , were so extract using sklearn 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')


In [ ]:
# vectors=cv.fit_transform(new_df['tags']).toarray().shape
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
vectors[0]

In [ ]:
# len(cv.get_feature_names_out()) O/P -> 5000
features_name=cv.get_feature_names_out()
features_name=pd.DataFrame(features_name)
features_name

In [ ]:
stem('In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

### Calculate distance of 1 movie's vector from 4806 movies's vector ony by one perform on all

In [ ]:
# cosine_similarity(vectors).shape -->o/p-- (4806,4806)
similarity=cosine_similarity(vectors)
# simirarity[0].shape

In [ ]:
# sorted(simirarity[0])[-10 : -1]
# sorted(simirarity[0],reverse=True)
# sorted(list(enumerate(simirarity[0])),reverse=True)
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
similarity[0] ## 1 movie ka sabhi movie ke sath similarity

In [ ]:
def recommend(movie):
    movie_index =new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index] 
    movies_list= sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        
    

In [ ]:
new_df.head(10)

In [ ]:
new_df[new_df['title']== 'John Carter'].index[0]

In [ ]:
recommend('Spider-Man 3')

In [ ]:
new_df.iloc[1216].title

# To deploy this project

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
new_df['title'].values

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
new_df.to_dict().head()